In [522]:
import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from gensim import corpora

In [523]:
with open("./sents.txt", "r") as f:
    sents = f.read().split("\n")
print(sents)

['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 

In [524]:
news1 = """ 국내 신종 코로나바이러스 감염증(코로나19) 신규 확진자가 하루만에 600명대로 올라섰다.전국적으로 크고 작은 일상 감염이 잇따르는 상황에서 해외유입 변이 바이러스도 빠르게 확산되고 있어 당국이 촉각을 세우고 있다. 정부는 오는 21일 다음주부터 적용할 사회적 거리두기 조정안을 발표할 예정이다.19일 중앙방역대책본부(중대본)에 따르면 이날 0시 기준 코로나19 신규 확진자는 654명으로 전날(528명)보다 126명 늘었다.누적 확진자는 13만3471명이다.확진자 수는 보통 주말·휴일 검사건수 감소 영향으로 인해 주 초반에는 비교적 적게 나오다가 중반부터 증가하는 경향을 보인다.이날 신규 확진자의 감염경로는 지역발생이 637명, 해외유입이 17명이다.지역별로는 서울 245명, 경기 159명, 인천 23명 등 수도권이 427명(67%)이다.주요 집단발병 사례를 보면 노래연습장, 유흥업소 등 다중이용시설을 고리로 새로운 감염이 확인됐다.수도권에서는 서울 강동구 노래연습장과 관련해 41명, 서울 노원구 고시원에서 11명, 경기 성남시 일가족-지인 사례에서 19명이 각각 양성 판정을 받았다.앞서 발생한 인천국제공항 검역소 집단감염과 관련해선 8명이 인도 변이에 감염된 것으로 확인됐다. 역학적 관련성이 있는 나머지 7명까지 포함하면 15명 전원이 인도 변이 감염자인 셈이다.최근 코로나19 발생 양상을 보면 전국에서 집단감염이 잇따르면서 '4차 유행'이 지속하고 있다.지난 13일부터 이날까지 최근 1주일간 하루 평균 신규 확진자는 약 651명꼴이다. 이중 '사회적 거리두기' 단계 조정의 핵심 지표인 일평균 지역발생 확진자는 약 628명으로, 여전히 2.5단계(전국 400∼500명 이상 등) 범위다.사망자는 전날보다 8명 늘어 누적 1912명이 됐다. 국내 평균 치명률은 1.43%다.전날 하루 선별진료소를 통한 검사 건수는 3만3640건으로, 직전일(4만1704건)보다 19% 적다.검사건수 대비 확진자를 계산한 양성률은 1.94%(3만3640명 중 654명)로, 직전일 1.27%(4만1704명 중 528명)보다 상승했다.정부는 오는 21일 다음주부터 3주간 적용할 사회적 거리두기 조정안을 발표한다. 현재 수도권에서는 2단계, 비수도권에서는 1.5단계의 거리두기 조처가 이뤄지고 있다."""

In [525]:
news2 = """경북 영천에서 17일 하루 만에 신종 코로나바이러스 감염증(코로나19) 확진자가 5명이 발생, N차 감염이 우려되는 가운데 비상이 걸렸다. 지난 7일 80번 확진자 이후 열흘 만에 지역 누적 확진자는 85명으로 늘어났다.영천시에 따르면 경산에 주소를 둔 공부방 교사가 확진됐다는 소식을 접한 A여고 한 학생이 지난 16일 보건소 검체 검사 결과 17일 양성판정을 받았다.또 확진자 교사가 운영 중인 공부방에서 과외 학습을 받고 있는 B여고 학생이 17일 오후 8시 확진 판정을 받은 가운데 A여고 학생 부모와 B여고 학생 엄마 등 총 5명이 코로나 판정을 받았다.수업을 받은 나머지 학생들은 모두 음성 판정을 받은 것으로 알려졌다.이날 A여고 측은 학생, 교직원 등 140여 명이 보건소에서 검체를 실시했으며 뒤늦게 알려진 B여고는 18일 학생과 교직원들이 검사를 받을 예정이다.이에 보건소는 코로나 확진자들을 자가 격리하고 이들의 이동 동선 파악에 집중하는 가운데 학교 등의 방역소독을 철저히 하고 있다."""


In [526]:
news3 =""" 브라질, 인도발 코로나 변이 상륙할라…긴장 속 모니터링 강화 (상파울루=연합뉴스) 김재순 특파원 = 브라질 보건당국이 인도발 신종 코로나바이러스 감염증(코로나19) 변이 바이러스 상륙 가능성에 긴장하고 있다.18일(현지시간) 브라질 언론에 따르면 상파울루시 당국은 인도발 변이 바이러스가 브라질에서도 나타날 가능성이 크다는 지적에 따라 상파울루주 정부 산하 부탄탕연구소와 함께 모니터링을 강화하기로 했다고 밝혔다.상파울루시는 브라질에서 코로나19 확진자와 사망자가 가장 먼저 나온 곳으로, 인근 과룰류스 국제공항을 통해 외국인 입국이 대규모로 이뤄지고 있다.상파울루시 관계자는 이날 기자회견을 통해 "인도발 변이 바이러스 상륙을 막기 위한 사전 조치로 부탄탕연구소와 협력해 외국인에 대한 철저한 추적 관찰이 이뤄질 것"이라고 말했다."""


In [527]:
news4 = """ 코로나 방역 모범국으로 평가받던 대만, 베트남, 싱가포르에서 최근 코로나 확진자가 크게 늘면서 방역에 비상이 걸렸다. 결국 백신 접종이 이뤄지지 않으면 코로나 확산세를 억제하기는 어렵다는 지적이 나온다. 미 블룸버그통신은 대만과 싱가포르가 자국내 방역 단계를 빠르게 격상시키고 있다고 16일 보도했다. 대만 정부는 이번 주말에 시민들이 집에서 머무르도록 권고하고, 실내 모임은 최대 5명, 야외 모임은 10명까지 제한했다. 국제 통계사이트 월드오미터 기준 대만에서 16일 보고된 코로나 신규 확진자는 207명으로 최고치를 기록했다.싱가포르 역시 식당 내에서의 식사를 금지하고 재택근무를 의무화했으며, 사적 모임은 최대 2인까지로 제한했다. 싱가포르 정부가 이런 봉쇄 정책을 실시한 것은 지난해 4~5월 이후 1년 만이다. 16일 보고된 싱가포르 신규 확진자는 49명이지만, 5월 초부터 집단 감염이 발생하자 경계 수위를 높였다. 또 최근 21일간 대만 여행 이력이 있는 외국인의 입국을 금지했다.대만과 싱가포르는 지난 12월 양국 간의 여행 제한 조치를 완화했었다. 양국에서 서로 체류하다 온 이들에 대해 자가격리 절차를 면제해 주기로 한 것이다. 하지만 자국 내 코로나 확산세가 심해지면서 여행객들에 대한 빗장도 다시 걸어 잠갔다.베트남은 이날 월드오미터 기준 127명의 신규 확진자가 보고됐다. 그 전날에는 165명의 확진자가 보고됐는데, 이는 지난해 1월 베트남에서 코로나가 처음 발병한 이래 최고치다. 북부 바짱주의 꽝차우 산업단지에서 집단감염이 일어나면서 확진자 수가 치솟았다.블룸버그는 이런 현상에 대해 “지역 사회에 대한 백신 접종 없이 바이러스로부터 안전한 환경을 만드는 게 얼마나 어려운지를 보여주는 사례”라고 했다. 블룸버그에 따르면 대만의 백신 접종률은 1% 미만이고, 베트남 역시 백신 접종률은 인구의 1% 수준이다. 싱가포르는 발빠르게 백신 확보에 나선 나라 중 하나이지만, 지금까지 한번이라도 백신을 맞은 국민은 전체 인구의 3분의 1 정도다."""


In [528]:
#news = [news1,news2,news3,news4]
#sents=news

In [529]:
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()
    return tokenizer

In [530]:
def get_tokens(sent: List[str], noun=True, tokenizer="mecab") -> List[str]:
    tokenizer = get_tokenizer(tokenizer)

    if noun:
        nouns = tokenizer.nouns(sent)
        print('n',nouns)
        nouns = [word for word in nouns if len(word) > 1]
        print('nouns',nouns)
        return nouns

In [531]:
#stopwords = ["연합뉴스", "가방"]

#vectorizer = CountVectorizer(
    #stop_words=stopwords,
    #tokenizer=partial(get_tokens, noun=True, tokenizer="mecab"),
    #min_df=2,
#)
# 코퍼스로부터 각 단어의 빈도 수를 기록.
#x = vectorizer.fit_transform(sents)
#print('x',x)
#x.toarray().shape
#print('x',x.toarray())


# 각 단어의 인덱스가 어떻게 부여되었는지를 보여줌.
#vocab_idx = vectorizer.vocabulary_
#print('v',vocab_idx)
#idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
#print(idx_vocab)

In [532]:
def vectorize_sents(
    sents: List[str], stopwords=None, min_count=2, tokenizer="mecab", noun=True
):
   

    vectorizer = CountVectorizer(
        stop_words=stopwords,
        tokenizer=partial(get_tokens, noun=noun, tokenizer="mecab"),
        min_df=min_count,
        #ngram_range=(1,2)
    )
    #vectorizer = CountVectorizer(
        #stop_words=stopwords,
        #tokenizer=partial(get_tokens, noun=noun, tokenizer="mecab"),
        #min_df=min_count,
        #ngram_range=(1, 2)
    #)


    vec = vectorizer.fit_transform(sents)
    print(vec)
    vocab_idx = vectorizer.vocabulary_
    print(vocab_idx)
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    print(idx_vocab)
    
    return vec, vocab_idx, idx_vocab

In [533]:
#stopwords = ["연합뉴스", "가방"]
#x, vocab_idx, idx_vocab = vectorize_sents(sents, stopwords)

#print('x',x)


In [534]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

In [535]:
#mat = word_similarity_matrix(x)

In [536]:
def word_graph(
    sents: List[str],
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
    stopwords: List[str] = ["연합뉴스", "그리고", "기자"],
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        sents, stopwords, min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

In [537]:
mat, vocab_idx, idx_vocab = word_graph(sents)
print('m',mat)
print('v',vocab_idx)
print('i',idx_vocab)


n ['기자']
nouns ['기자']
n ['연합뉴스']
nouns ['연합뉴스']
n []
nouns []
n ['오', '패산', '터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자', '일', '오후', '서울', '강북구', '오', '패산', '터널', '인근', '사제', '총기', '발사', '경찰', '살해', '용의자', '모씨', '검거', '성', '씨', '검거', '당시', '서바이벌', '게임', '방탄조끼', '헬멧', '착용', '상태']
nouns ['패산', '터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자', '오후', '서울', '강북구', '패산', '터널', '인근', '사제', '총기', '발사', '경찰', '살해', '용의자', '모씨', '검거', '검거', '당시', '서바이벌', '게임', '방탄조끼', '헬멧', '착용', '상태']
n ['서울', '연합뉴스', '김은경', '기자', '사제', '총기', '경찰', '살해', '범인', '성모', '씨']
nouns ['서울', '연합뉴스', '김은경', '기자', '사제', '총기', '경찰', '살해', '범인', '성모']
n ['경찰', '성', '씨', '일', '오후', '강북', '경찰서', '인근', '부동산', '업소', '밖', '부동산', '업자', '이모', '씨', '이', '씨', '평소', '말다툼', '자주', '것']
nouns ['경찰', '오후', '강북', '경찰서', '인근', '부동산', '업소', '부동산', '업자', '이모', '평소', '말다툼', '자주']
n ['씨', '시작', '성', '씨', '준비', '사제', '총기', '이', '씨', '발사', '총알', '이', '씨', '총알', '행인', '씨', '배']
nouns ['시작', '준비', '사제', '총기', '발사', '총알', '총알', '행인']
n [

In [538]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

In [539]:

#def pagerank(x, df=0.85, max_iter=30,method="iterative"):
    #assert 0 < df < 1
    #if method == "iterative":
        # initialize
        #A = normalize(x, axis=0, norm='l1')
        #R = np.ones(A.shape[0]).reshape(-1,1)
        #bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

        # iteration
        #for _ in range(max_iter):
           # R = df * (A * R) + bias

   # return R

In [540]:
R = pagerank(mat,method ="iterative")

In [541]:
topk = 10
idxs = R.argsort()[-topk:]


In [542]:
keywords = [(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)]
keyword = [(R[idx]) for idx in reversed(idxs)]

In [543]:
keywords

[(20, 0.038171474706220364, '서울'),
 (10, 0.03590352211807886, '발사'),
 (28, 0.03567521486237227, '용의자'),
 (2, 0.03458053216612935, '강북구'),
 (17, 0.03455531359160814, '사제'),
 (29, 0.034091948403762985, '의자'),
 (33, 0.034091948403762985, '조사'),
 (5, 0.034091948403762985, '경찰관'),
 (34, 0.03328840487152944, '총기'),
 (39, 0.03296229487665027, '폭행')]

In [498]:
keywords

[(33, 0.022927395999148576, '이날'),
 (48, 0.02234111218256318, '코로나'),
 (54, 0.021844150916388954, '확진'),
 (39, 0.02156335352608398, '정부'),
 (42, 0.020912371095318485, '지역'),
 (15, 0.0203363937910987, '발생'),
 (0, 0.02026975697232176, '감염'),
 (44, 0.020128252109039495, '집단'),
 (23, 0.02009101181436368, '신규'),
 (24, 0.02009101181436368, '신규 확진')]

In [499]:
keyword

[0.022927395999148576,
 0.02234111218256318,
 0.021844150916388954,
 0.02156335352608398,
 0.020912371095318485,
 0.0203363937910987,
 0.02026975697232176,
 0.020128252109039495,
 0.02009101181436368,
 0.02009101181436368]